In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
#표를 불러오기 위한 라이브러리
from html_table_parser import parser_functions as parser
import collections
collections.Callable = collections.abc.Callable
#403 에러 방지
headers={"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"}

In [2]:
#csv 파일 읽기

file_name = '반팔'+'_url_list.csv'
data = pd.read_csv(file_name, names=['name', 'url'])
urls=data['url'].values
print(urls.shape, urls)

(8911,) ['url' 'https://www.musinsa.com/app/goods/2368712'
 'https://www.musinsa.com/app/goods/2427585' ...
 'https://www.musinsa.com/app/goods/2000501'
 'https://www.musinsa.com/app/goods/1486747'
 'https://www.musinsa.com/app/goods/3504350']


In [6]:
df=pd.DataFrame(data, columns=['name', 'url'])
url_list=df.values.tolist()

print(url_list)

[[nan, 'url'], [0.0, 'https://www.musinsa.com/app/goods/2368712'], [1.0, 'https://www.musinsa.com/app/goods/2427585'], [2.0, 'https://www.musinsa.com/app/goods/3476114'], [3.0, 'https://www.musinsa.com/app/goods/903340'], [4.0, 'https://www.musinsa.com/app/goods/2029969'], [5.0, 'https://www.musinsa.com/app/goods/3314009'], [6.0, 'https://www.musinsa.com/app/goods/1482975'], [7.0, 'https://www.musinsa.com/app/goods/3294589'], [8.0, 'https://www.musinsa.com/app/goods/3555410'], [9.0, 'https://www.musinsa.com/app/goods/540421'], [10.0, 'https://www.musinsa.com/app/goods/3382679'], [11.0, 'https://www.musinsa.com/app/goods/2028864'], [12.0, 'https://www.musinsa.com/app/goods/3363176'], [13.0, 'https://www.musinsa.com/app/goods/1816729'], [14.0, 'https://www.musinsa.com/app/goods/2329824'], [15.0, 'https://www.musinsa.com/app/goods/3341803'], [16.0, 'https://www.musinsa.com/app/goods/1884943'], [17.0, 'https://www.musinsa.com/app/goods/1944612'], [18.0, 'https://www.musinsa.com/app/goods/3

In [11]:
url=url_list[1][1]

page = requests.get(url, headers = headers)
soup = bs(page.text, 'lxml')
table = soup.find_all(attrs={'class':'table_th_grey'})

#print(table)

p = parser.make2d(table[0])
df = pd.DataFrame(p[3:], columns=p[0])
print(df)

   cm  총장  어깨너비  가슴단면 소매길이
0  XS  58    44    50   19
1   S  67    48    52   22
2   M  70  50.5  55.5   24
3   L  72    52    58   25
4  XL  74  53.5  60.5   26
